In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
sys.version
import datetime
import dateutil.parser
import json
import joblib

#学習用データ，検証用データの読み込み先とそれぞれのdataframeの保存先に注意
forLearning = 1#学習用データなら1，検証用は0



if forLearning == 1:
    with open("/data/label-for-learning.json", "r") as f:
        df_all = pd.read_json(f)#学習用ラベル
elif forLearning == 0:
    with open("/data/label-for-evaluation.json", "r") as f:
        df_all = pd.read_json(f)#検証用評価データ
else:
    print('choose correct mode')

    
#dataframeのtypeが一致するかどうか確認する関数
def type_check(df1,df2):
    if df1['type'][0] == df2['type'][0]:
        return True
    else:
        return False

#type_check(df[1],df[3])

#typeの一致したdataframeを結合する関数
def merge(df1,df2):
    if type_check(df1,df2)==True:
        temp=pd.concat([df1,df2])
        df1=pd.DataFrame(columns=cols)
        df2=temp
    return df1,df2

    
    
    
#recipes毎にデータフレーム作成
recipes = df_all['recipes']
#recipesの中身でデータフレーム作成を確認
#print('df_allの時刻：',df_all['started_at'][0])#JST
#print(recipes[0]['started_at'])#UTCっぽいのでこれをJSTに合わせる，つまり9時間足す

#recipesの中身でデータフレーム作成，時刻表記を変更
df={}
for i in range(len(df_all)):
    df[i]=pd.DataFrame(recipes[i],index=[0,])
    temp_start=pd.to_datetime(df[i]['started_at'][0])#時刻表記datetime型に
    df[i].loc[0,'started_at']=temp_start+datetime.timedelta(hours=9)#時差処理
    temp_stop=pd.to_datetime(df[i]['stopped_at'][0])
    df[i].loc[0,'stopped_at']=temp_stop+datetime.timedelta(hours=9)
    
#indexをrecipesのstarted_atからstopped_atで作成してDF作成，recipesの中身をそのまま貼り付け．
#isFalureの列を作成，recipes時間内は異常時なので全てyes
cols = ['isFalure','message','network','node','project','started_at','status','stopped_at','time','type']
for i in range(len(df_all)):
    dates = pd.date_range(start=df[i]['started_at'][0],end=df[i]['stopped_at'][0],freq='S')
    temp = pd.DataFrame(index=dates,columns=cols)
    temp2 = pd.concat([df[i],temp])
    temp2.loc[0,'isFalure']=True
    temp2.loc[:,:]=temp2.loc[0].values
    temp2=temp2.drop(0,axis=0)
    df[i] = temp2


#同じtypeのDFがあればmerge
check=pd.DataFrame()
n=0
for i in range(len(df_all)-1):#0-1861まであり，1860と1861の比較が最後になる
    n=0
    n=i+1#df[i]の1つ次のdataframeから比較を始める
    while type_check(df[i],df[n])==False:      
        n=n+1
        if n>=(len(df_all)-1):#nがdf数を超えるとまずい
            n=(len(df_all)-1)
            #print(n)
            break
    df[i],df[n] = merge(df[i],df[n])
    if df[i].empty==True:#typeが一致してmergeしたものはその番号を記録
        forcheck = [i,n]
        checkS=pd.Series(data=forcheck)
        check=check.append(checkS,ignore_index=True)
#checkを確認すればどのdataframeがどのdataframeにmergeされたかがわかる
#check.head()

#typeをdataframeの名前にする
#namesにどのようなtypeの種類があるか記録する，14種類出てきた
df_names=pd.DataFrame(columns=['type'])
for i in range(len(df_all)):#0-1861 in range(1862)
    if df[i].empty == False:
        temp = df[i]['type'][0]
        #print('df[%d]:found'% i)
        #print(temp)
        add = {'type':temp}
        df[temp]=pd.DataFrame()
        df[temp]=df[i]
        df_names=df_names.append(add,ignore_index=True)
        
#df_allでのstarted_atやstopped_atに複数種類あるのかどうか確認：ない，全部同時刻
#ここが揃っていないと厄介なので注意
for i in range(len(df_all)):#0-1861 in range(1862)
    if df_all['status'][i]!='succeeded':
        print(i)
    if df_all['started_at'][i]!=df_all['started_at'][0]:
        print('start time changed %' % i)
    if df_all['stopped_at'][i]!=df_all['stopped_at'][0]:
        print('stop time changed %' % i)

#df_allの列名を取得してそれでdataframe作成
df_base=pd.DataFrame(columns=df_all.columns,index=[0,])
df_base.iloc[0]=df_all.iloc[0]
#df_base.head()


#started_atの一番早い時刻を選ぶ
time = df_all['started_at'][0]
for i in df_names.type:
    if time > df[i]['started_at'][0]:
        time = df[i]['started_at'][0]

#df_allのstartstop時刻はすべて共通なので，全時刻をindexにした共通dataframeを作成
cols.extend(df_all.columns)#colsにdf_allの列結合
cols4base = set(cols)#重複削除
#dates_base = pd.date_range(start=df_all['started_at'][0],end=df_all['stopped_at'][0],freq='S')
dates_base = pd.date_range(start=time,end=df_all['stopped_at'][0],freq='S')
temp_base = pd.DataFrame(index=dates_base,columns=cols4base)
temp_base2 = pd.concat([df_base,temp_base])
temp_base2.loc[0,'isFalure']=False
temp_base2.loc[:,:]=temp_base2.loc[0].values
temp_base2=temp_base2.drop(0,axis=0)
df_base = temp_base2
#df_base.head()

#時刻範囲がdf_allとrecipesのstarted_at,stopped_atでどうなっているか確認
#startもstopもずれているとdf_baseと結合したときにindexがとんでしまう可能性があるがどうやらそれはなさそう．
for i in df_names.type:
    #print(i)
    if df[i].started_at[0]<df_all.started_at[0]:
        print('start early',i)
    if df[i].stopped_at[0]>df_all.stopped_at[0]:
        print('stop late',i)
    if df[i].stopped_at[0]<df_all.started_at[0]:
        print('stop before df_all start',i)
    if df[i].started_at[0]>df_all.stopped_at[0]:
        print('start after df_all stop',i)
    if df[i].started_at[0]>=df_all.started_at[0] and df[i].stopped_at[0]<=df_all.stopped_at[0]:
        print('in time range of df_all',i)
    else:
        print('something wrong',i)
        print(df[i].started_at[0])
        print(df[i].stopped_at[0])

#各データフレームと作成した全時刻データフレームdf_baseを合体し，時刻でソート
for i in df_names.type:
    #recipesのstarted_atからstopped_atまでの時刻分の行を削除
    tempIndex_df = df_base
    indexNames = tempIndex_df[(tempIndex_df.index>=df[i]['started_at'][0]) & (tempIndex_df.index<=df[i]['stopped_at'][len(df[i])-1])].index
    tempIndex_df.drop(indexNames, inplace=True)
    #reset_index()で現在のindexを新たに生成したindex列に移し，concat
    temp_base = tempIndex_df.reset_index()
    temp_df = df[i].reset_index()
    temp_final = pd.concat([temp_base,temp_df],ignore_index=True)
    temp_reindex = temp_final.set_index('index')
    df[i] = temp_reindex.sort_index()
    if forLearning == 1:#学習用データ保存
        df[i].to_pickle("/data/output-directory_all/df/label/{0}.pkl".format(i))
    elif forLearning == 0:#検証用データ保存
        df[i].to_pickle("/data/output-directory_random/df/label/{0}.pkl".format(i))
    else:
        print('something is wrong')